In [1]:
import sys
sys.path.append("../")

import pandas as pd
import os
import statsmodels.formula.api as sm
import engarde.decorators as ed

from library import regulations
from library import characteristics
from library import analysis
from library import tables
from library import start

In [8]:
data_path = start.data_path
table_path = start.table_path
data = pd.read_csv(os.path.join(start.data_path, 'clean', 'master_data_district.csv'),
                  sep=",", low_memory = False)
data = data[data.year == 2017]
data = data[data.distischarter == "N"]
print(len(data[data.doi == True]))
print(len(data[data.doi == False]))

822
201


## District Characteristics

In [9]:
# Traditional public schools versus DOI
district = analysis.many_y_one_x(data = data, 
                                 y_list = characteristics.geography,
                                 y_labels = characteristics.labels,
                                 x = 'doi')
district

type_urban ~ + 1 + doi
type_suburban ~ + 1 + doi
type_town ~ + 1 + doi


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


type_rural ~ + 1 + doi


,Characteristic,Control,Difference,Std. Error,P-value
0,Urban,0.06,-0.01,0.02,0.38
1,Suburban,0.19,0.05,0.03,0.11
2,Town,0.25,0.01,0.03,0.71
3,Rural,0.49,-0.05,0.04,0.18


## Teacher Characteristics

In [10]:
teacher = analysis.many_y_one_x(data = data, 
                                 y_list = characteristics.teacher,
                                 y_labels = characteristics.labels,
                                 x = 'doi')
teacher

teachers_exp_ave ~ + 1 + doi
teachers_tenure_ave ~ + 1 + doi
teachers_turnover_ratio_d ~ + 1 + doi
students_teacher_ratio ~ + 1 + doi


,Characteristic,Control,Difference,Std. Error,P-value
0,Ave. Experience Teaching,11.73,0.71,0.20,0.00
1,Ave. Tenure in District,6.72,0.35,0.16,0.03
2,Teacher Turnover Ratio,21.10,-2.36,0.72,0.00
3,Student-Teacher Ratio,12.48,0.29,0.19,0.13


## Student Characteristics

In [11]:
student = analysis.many_y_one_x(data = data, 
                                 y_list = characteristics.student,
                                 y_labels = characteristics.labels,
                                 x = 'doi')
student

students_hisp ~ + 1 + doi
students_white ~ + 1 + doi
students_black ~ + 1 + doi
students_frpl ~ + 1 + doi
avescores ~ + 1 + doi


,Characteristic,Control,Difference,Std. Error,P-value
0,Percent Hispanic,0.53,-0.17,0.02,0.00
1,Percent White,0.38,0.15,0.02,0.00
2,Percent Black,0.06,0.01,0.01,0.19
3,Percent Econ. Disadvantaged,0.66,-0.10,0.01,0.00
4,Average STAAR Performance (Std.),-0.10,0.41,0.06,0.00



## To table

In [12]:
dfs = [district, teacher, student]
rows = [6, 15, 24]
tables.n_to_excel(file = table_path + 'balance_tpsVdoi.xlsx',
                 col = 2, row = 4, n = len(data[data.doi == 0]))
tables.n_to_excel(file = table_path + 'balance_tpsVdoi.xlsx',
                 col = 3, row = 4, n = len(data[data.doi == 1]))
for df, row in zip(dfs,rows):
    tables.var_diff_to_excel(file = table_path + 'balance_tpsVdoi.xlsx', 
                             df = df, 
                             control_col = 'Control', 
                             diff_col = 'Difference', 
                             se_col = 'Std. Error', pvalue_col = 'P-value',
                             start_col = 2, start_row = row)